<a href="https://colab.research.google.com/github/yshin1209/adll/blob/main/notebooks/Aura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install neo4j

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [19]:
from neo4j import GraphDatabase as graphdb

In [25]:
uri = "neo4j+s://34b10577.databases.neo4j.io"
user = "neo4j"
password = "A6uVT1wvZfAFLE_O9ilqxiHuDV30in887nHF-ByS_W0"

In [26]:
class Graph:
    # Initialize graph database class
    def __init__(self, grapdb, uri, user, password):
        self.driver = graphdb.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    # Add node
    @staticmethod
    def _add_node_tx(tx, node_label, key, value):
        query = "MERGE (n:" + node_label + "{ " + key + ": $value })" \
                " RETURN n{.*}" # Return all property key value pairs
        result = tx.run(query, value = value)
        return result.single()[0]

    def add_node(self, node_label, key, value):
        with self.driver.session() as session:
            result = session.execute_write(self._add_node_tx, node_label, key, value)
            return result

    # Set node property value
    @staticmethod
    def _set_node_prop_tx(tx, id_key, id_value, key, value):
        query = "MATCH (n) WHERE n." + id_key + " = $id_value" \
                " SET n." + key + " = $value" \
                " RETURN n"
        result = tx.run(query, id_value = id_value, value = value)
        return result.single()[0]

    def set_node_prop(self, id_key, id_value, key, value):
        with self.driver.session() as session:
            result = session.execute_write(self._set_node_prop_tx, id_key, id_value, key, value)
            return result
    
    # Get node property value
    @staticmethod
    def _get_node_prop_tx(tx, id_key, id_value, key ):
        query = "MATCH (n) WHERE n." + id_key + " = $id_value" \
                " RETURN n." + key + " as value"
        result = tx.run(query, id_value = id_value)
        return result.single()[0]

    def get_node_prop(self, id_key, id_value, key):
        with self.driver.session() as session:
            result = session.execute_write(self._get_node_prop_tx, id_key, id_value, key)
            return result



In [28]:
g = Graph(graphdb, uri, user, password)

In [30]:
print (g.add_node('Person', 'id', 'YongJUN'))

{'id': 'YongJUN'}


In [7]:
g.set_node_prop('name', 'YongJUN', 'age', 19)

<Node element_id='4:39b4fa34-f736-4657-87d5-9ea4e101fac4:7' labels=frozenset({'Person'}) properties={'name': 'YongJUN', 'age': 19}>

In [ ]:
g.get_node_prop('name', 'YongJUN', 'name')

'YongJUN'

In [ ]:
# Add node
def add_node_tx(tx, node_label, key, value):
    query = "MERGE (:" + node_label + "{ " + key + ": $value })"
    tx.run(query, value = value)
def add_node(label, key, value):
    with driver.session() as session:
        session.execute_write(add_node_tx, label, key, value)

# Get node property value
def get_node_prop_tx(tx, id_key, id_value, key ):
    query = "MATCH (n) WHERE n." + id_key + " = $id_value" \
            " RETURN n." + key + " as value"
    tx.run(query, id_value = id_value)
    for record in result:
      return (record["value"])
def get_node_prop(key1, value1, key2):
    with driver.session() as session:
        result = session.execute_write(get_node_prop_tx, key1, value1, key2)
        return result

# Update node property value
def update_node_prop_tx(tx, id_key, id_value, key, value):
    query = "MATCH (n) WHERE n." + id_key + " = $id_value" \
            " SET n." + key + " = $value" 
    tx.run(query, id_value = id_value, value = value)
def update_node_prop(id_key, id_value, key, valu):
    with driver.session() as session:
        session.execute_write(update_node_prop_tx, id_key, id_value, key, valu)

# Update edge property value
def update_edge_prop_tx(tx, out_key, out_val, in_key, in_val, edge_key, edge_val):
    query = "MATCH (n) WHERE n." + out_key + " = $out_val" \
            " MATCH (m) WHERE m." + in_key + " = $in_val" \
            " MATCH (n)-[r]->(m)" \
            " SET r." + edge_key + " = $edge_val" 
    tx.run(query, out_val = out_val, in_val = in_val, edge_val = edge_val)
def update_edge_prop(out_key, out_val, in_key, in_val, edge_key, edge_val):
    with driver.session() as session:
        session.execute_write(update_edge_prop_tx, out_key, out_val, in_key, in_val, edge_key, edge_val)

# Add edge
def add_edge_tx(tx, edge_label, out_key, out_val, in_key, in_val):
    query = "MATCH (n) WHERE n." + out_key + " = $out_val" \
            " MATCH (m) WHERE m." + in_key + " = $in_val" \
            " MERGE (n)-[r:" + edge_label + "]->(m)"
    results = tx.run(query, out_val = out_val, in_val = in_val).data
    return results
def add_edge(edge_label, out_key, out_val, in_key, in_val):
    with driver.session() as session:
        results = session.execute_write(add_edge_tx, edge_label, out_key, out_val, in_key, in_val)
        print (results)

# Delete edge
def delete_edge_tx(tx, edge_label, out_key, out_val, in_key, in_val):
    query = "MATCH (n) WHERE n." + out_key + " = $out_val" \
            " MATCH (m) WHERE m." + in_key + " = $in_val" \
            " MATCH (n)-[r:" + edge_label + "]->(m)" \
            " DELETE r" 
    tx.run(query, out_val = out_val, in_val = in_val)
def delete_edge(edge_label, out_key, out_val, in_key, in_val):
    with driver.session() as session:
        session.execute_write(delete_edge_tx, edge_label, out_key, out_val, in_key, in_val)

# Delete all
def delete_all_tx(tx):
    query = "MATCH (n) DETACH DELETE n"
    tx.run(query)
def delete_all():
    with driver.session() as session:
        session.execute_write(delete_all_tx)

# Execute math operation
def math_op_tx(tx, key, value):
    query = "MATCH (x) WHERE x." + key + " = $value" \
            " MATCH (x) -[:D2OP]-> (op) <-[:P2OP]-(w)" \
            " WITH op," \
            " CASE op.name" \
            " WHEN 'add' THEN x.value+w.value" \
            " WHEN 'mul' THEN x.value*w.value" \
            " END AS result" \
            " MATCH (op)-[:OP2D]->(y)" \
            " SET y.value = result" \
            " RETURN y.name as name"
    result = tx.run(query, value = value)
    for record in result:
      return (record["name"])
def math_op(key, value):
    with driver.session() as session:
        result = session.execute_write(math_op_tx, key, value)
    return result

In [ ]:
add_edge('state_transition', 'name', 's3', 'name', 's4')

<bound method Result.data of <neo4j._sync.work.result.Result object at 0x7f7fcc85f810>>


In [ ]:
delete_edge('D2OP', 'name', 'x', 'name', 'mul')

In [ ]:
update_node_prop('name', 's3', 'reward', 0)

In [ ]:
get_node_prop('name', 's2', 'reward')

-100

In [ ]:
update_edge_prop('name', 's1', 'name', 's3',  'name', 'st2')

In [ ]:
add_node('State', 'name', 's4')

In [ ]:
delete_all()

In [ ]:
result = math_op('name', 'x')
print (result)
result2 = math_op('name', result)
print (result2)

y
y2


In [ ]:
result = math_op('name', 'x')
while True:
  result = math_op('name', result)
  if result == None:
    break

In [ ]:
N = 1000
for age in range(1,N):
  add_node('Person', 'age', age)

In [ ]:
#name2 = 'Jun'
# Cypher query
query = """
  match (n:Person {name: $name1})
  match (m:Person {name: 'So'})
  merge (n)-[r:knows]-> (m)
  return n, r, m
"""

# Create a driver session
with driver.session() as session:
    # Use .data() to access the results array
    results = session.run(query, 'Jun2').data()
    print(results)

ClientError: ignored

In [ ]:
# Search all nodes
for key, value in results[0].items():
  print (key, value)

n {'name': 'Jun'}
r ({'name': 'Jun'}, 'knows', {'name': 'So'})
m {'name': 'So'}


In [ ]:
def create_person(tx, name):
    tx.run("CREATE (a:Person {name: $name})", name=name)

def create_friend_of(tx, name, friend):
    tx.run("MATCH (a:Person) WHERE a.name = $name "
           "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
           name=name, friend=friend)

with driver.session() as session:
    session.execute_write(create_person, "Alice")
    session.execute_write(create_friend_of, "Alice", "Bob")
    session.execute_write(create_friend_of, "Alice", "Carl")

driver.close()

In [ ]:
def get_friends_of(tx, name):
    friends = []
    result = tx.run("MATCH (a:Person)-[:KNOWS]->(f) "
                    "WHERE a.name = $name "
                    "RETURN f.name AS friend", name=name)
    for record in result:
        friends.append(record["friend"])
    return friends

with driver.session() as session:
    friends = session.execute_read(get_friends_of, "Alice")
    for friend in friends:
        print(friend)

driver.close()

Bob
Carl
